# Uploading your own deep zoom images

Once you have managed to register a Backblaze cloud storage account and created a public bucket, the hard work is done! 

Upload your deep zoom image by executing the following code in a Jupyter notebook:  

**Step 1** 

You need to copy your own personal B2 bucket application-key, application-key-id and bucket-name in a notebook cell and run the cell (by pressing Shift-Enter). 

```Python 
application_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxx" 
application_key_id = "yyyyyyyyyyyyyyyyyyyyyyyyyy"  
bucket_name = "my-first-bucket" 
```


**Step 2** 

Import the deepzoomup package and connect to your B2 bucket:

```Python 
import deepzoomup 
my_dzs = deepzoomup.DeepZoomStore(application_key_id, application_key, bucket_name) 
```

**Step 3** 

Select the image file and upload: 

```Python 
my_image = 'path/to/my/huge.tif'
my_dzs.make_deepzoom(my_image) 
```

You can repeat this for other images: 

```Python

my_image = 'path/to/another/huge.png' 
my_dzs.make_deepzoom(my_image)
```

**That is all!** 

Now your deep zoom image can be viewed on-line: *need-to-provide-weblink*.  

If you would like to create more advanced viewers for viewing multiple images simultaneously, read the next page. 